In [7]:
from flask import Flask, redirect, url_for, render_template, request,jsonify, make_response
import New_Bucket
import pandas as pd
from flask_cors import CORS
from sklearn.preprocessing import LabelEncoder
import numpy as np

data = pd.DataFrame()
app = Flask(__name__)
CORS(app)

class EncoderExt(object):
    def __init__(self):
        self.label_encoder = LabelEncoder()
    def fit(self, data_list):
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_
        return self
    def transform(self, data_list):
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]
        return self.label_encoder.transform(new_data_list)
    
def predict(data):
    
    print(data)
    # Clear Date
    data['clear_date'] = pd.to_datetime(data['clear_date'], format = '%Y-%m-%d')
    # Posting Date
    data['posting_date'] = pd.to_datetime(data['posting_date'], format = '%Y-%m-%d')
    # Due in Date
    data['due_in_date'] = pd.to_datetime(data['due_in_date'], format = '%Y-%m-%d')
    # Baseline Create Date
    data['baseline_create_date'] = pd.to_datetime(data['baseline_create_date'], format = '%Y-%m-%d')
    data['converted_usd'] = pd.to_numeric(data['converted_usd'])
     # buisness_year
    data['buisness_year'] = pd.to_datetime(data['buisness_year'], format = '%Y-%m-%d')
    data['buisness_year'] = data['buisness_year'].dt.year
    
    from sklearn.preprocessing import LabelEncoder
    business_codern = LabelEncoder()
    business_codern.fit(data['business_code'])
    data['business_code_enc'] = business_codern.transform(data['business_code'])
    data['cust_number'] = data['cust_number'].replace('CCCA02',"1").replace(['CCU001','CCU013','CCU002'],"2").replace(['CC3411','CC6000'],"3").astype(int)
    data['day_of_cleardate'] = data['clear_date'].dt.day
    data['month_of_cleardate'] = data['clear_date'].dt.month
    data['year_of_cleardate'] = data['clear_date'].dt.year

    data['day_of_postingdate'] = data['posting_date'].dt.day
    data['month_of_postingdate'] = data['posting_date'].dt.month
    data['year_of_postingdate'] = data['posting_date'].dt.year

    data['day_of_due'] = data['due_in_date'].dt.day
    data['month_of_due'] = data['due_in_date'].dt.month
    data['year_of_due'] = data['due_in_date'].dt.year

    data['day_of_createdate'] = data['baseline_create_date'].dt.day
    data['month_of_createdate'] = data['baseline_create_date'].dt.month
    data['year_of_createdate'] = data['baseline_create_date'].dt.year
    label_encoder = EncoderExt()
    label_encoder1 = EncoderExt()
    label_encoder1.fit(data['cust_payment_terms'])
    label_encoder1.fit(data['business_code'])
    label_encoder.fit(data['name_customer'])
    data['cust_payment_terms_enc']=label_encoder1.transform(data['cust_payment_terms'])
    data['business_code_enc']=label_encoder1.transform(data['business_code'])
    data['name_customer_enc']=label_encoder.transform(data['name_customer'])
    data.drop(['business_code',"baseline_create_date","due_in_date","posting_date","name_customer","clear_date","cust_payment_terms"],axis=1,inplace=True)
    data.drop(['day_of_cleardate',"month_of_cleardate","year_of_cleardate"],axis=1,inplace=True)
    data2=data[['cust_number', 'buisness_year', 'doc_id', 'converted_usd',
           'business_code_enc', 'name_customer_enc', 'cust_payment_terms_enc',
           'day_of_postingdate', 'month_of_postingdate', 'year_of_postingdate',
           'day_of_createdate', 'month_of_createdate', 'year_of_createdate',
           'day_of_due', 'month_of_due', 'year_of_due']]
    import pickle
    model = pickle.load(open("model.sav", 'rb'))
    final_result = model.predict(data2)
    final_result = pd.Series(final_result,name='avg_delay')
    data2.reset_index(drop=True,inplace=True)
    Final = data2.merge(final_result , on = data.sl_no )
    Final['avg_delay'] = Final.apply(lambda row: row.avg_delay//(24 * 3600), axis = 1)
    bins= [-np.inf, 0, 15, 30, 45, 60, 100]
    labels = ['<0','0-15','16-30','31-45','46-60','Greatar than 60']
    Final['Aging Bucket'] = pd.cut(Final['avg_delay'], bins=bins, labels=labels, right=False)
    Final.drop(['key_0',"avg_delay"],axis=1,inplace=True)
    Final1=Final[['doc_id','Aging Bucket']].copy()
    doc = Final1["doc_id"].tolist()
    aging= Final1["Aging Bucket"].tolist()   
    a=[]
    print(Final1)
    for i in range(len(doc)):
        a.append({"doc_id":doc[i],"aging_bucket": str(aging[i])})
    

    return a



@app.route("/predict",methods=["POST", "GET"])
def home():
    if request.method == "POST":
#         print(request.json)
        content = request.get_json()
        required_cols = ["sl_no", "business_code", "cust_number", "name_customer", "clear_date", "buisness_year", "doc_id", "posting_date", "due_in_date", "baseline_create_date", "cust_payment_terms", "total_open_amount"]
        rows = []
        
        for row in content:
            row_data = []
            for key, value in row.items():
                if key in required_cols:
                    row_data.append(value)
            rows.append(row_data)
#         print(rows)            
        data = pd.DataFrame(rows, columns=["sl_no", "business_code", "cust_number", "name_customer", "clear_date", "buisness_year", "doc_id", "posting_date", "due_in_date", "converted_usd", "baseline_create_date", "cust_payment_terms"])
        data.set_index('sl_no')
        
        data['doc_id'] = data['doc_id'].astype(int)
#         data['converted_usd'] = data['converted_usd'].astype(float)
#       print(data.info())
#       print(data)
#         print(predict(data))   
        response = make_response(jsonify(predict(data)),200)
        response.headers["Access-Control-Allow-Origin"]="*"
        response.headers["Content-Type"] = "application/json"
        return response
   
        
if __name__ == "__main__":
    app.run()
    



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2022 11:05:57] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\abhis\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
127.0.0.1 - - [19/Apr/2022 11:05:57] "POST /predict HTTP/1.1" 200 -


   sl_no business_code cust_number   name_customer  clear_date buisness_year  \
0      5          U001   200769623  PIO associates  2019-11-25    2019-01-01   

       doc_id posting_date due_in_date converted_usd baseline_create_date  \
0  1930147974   2019-11-13  2019-11-28      33133.29           2019-11-13   

  cust_payment_terms  
0               NAH4  


127.0.0.1 - - [19/Apr/2022 11:06:46] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\abhis\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
127.0.0.1 - - [19/Apr/2022 11:06:47] "POST /predict HTTP/1.1" 200 -


   sl_no business_code cust_number   name_customer  clear_date buisness_year  \
0      5          U001   200769623  PIO associates  2019-11-25    2019-01-01   

       doc_id posting_date due_in_date converted_usd baseline_create_date  \
0  1930147974   2019-11-13  2019-11-28      33133.29           2019-11-13   

  cust_payment_terms  
0               NAH4  


...

   sl_no business_code cust_number   name_customer  clear_date buisness_year  \
0      2          U001   200980828     SYS systems  2019-08-08    2019-01-01   
1      3          U001   200792734       SUPERB us  2019-12-30    2019-01-01   
2      5          U001   200769623  PIO associates  2019-11-25    2019-01-01   

       doc_id posting_date due_in_date converted_usd baseline_create_date  \
0  1929646410   2019-07-22  2019-08-11       79656.6           2019-07-22   
1  1929873765   2019-09-14  2019-09-29       2253.86           2019-09-14   
2  1930147974   2019-11-13  2019-11-28      33133.29           2019-11-13   

  cust_payment_terms  
0               NAH4  
1               NAH4  
2               NAH4  


127.0.0.1 - - [19/Apr/2022 11:13:22] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\abhis\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
127.0.0.1 - - [19/Apr/2022 11:13:22] "POST /predict HTTP/1.1" 200 -


   sl_no business_code cust_number   name_customer  clear_date buisness_year  \
0      1          U001   200769623  PIO associates  2020-02-11    2020-01-01   
1      2          U001   200980828     SYS systems  2019-08-08    2019-01-01   
2      3          U001   200792734       SUPERB us  2019-12-30    2019-01-01   
3      5          U001   200769623  PIO associates  2019-11-25    2019-01-01   

       doc_id posting_date due_in_date converted_usd baseline_create_date  \
0  1930438491   2020-01-26  2020-02-10      54273.28           2020-01-26   
1  1929646410   2019-07-22  2019-08-11       79656.6           2019-07-22   
2  1929873765   2019-09-14  2019-09-29       2253.86           2019-09-14   
3  1930147974   2019-11-13  2019-11-28      33133.29           2019-11-13   

  cust_payment_terms  
0               NAH4  
1               NAH4  
2               NAH4  
3               NAH4  


127.0.0.1 - - [19/Apr/2022 11:15:26] "OPTIONS /predict HTTP/1.1" 200 -
[2022-04-19 11:15:26,894] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\abhis\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\abhis\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\abhis\anaconda3\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\abhis\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\abhis\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\abhis\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_reque

   sl_no business_code cust_number   name_customer  clear_date buisness_year  \
0      1          U001   200769623  PIO associates  2020-02-11    2020-01-01   

       doc_id posting_date due_in_date converted_usd baseline_create_date  \
0  1930438491   2020-01-26  2020-02-10      54273.28           2020-01-26   

  cust_payment_terms  
0               NAH4  


In [10]:
from flask import Flask, redirect, url_for, render_template, request,jsonify, make_response
import New_Bucket
import pandas as pd
from flask_cors import CORS
from sklearn.preprocessing import LabelEncoder
import numpy as np

data = pd.DataFrame()
app = Flask(__name__)
CORS(app)

class EncoderExt(object):
    def __init__(self):
        self.label_encoder = LabelEncoder()
    def fit(self, data_list):
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_
        return self
    def transform(self, data_list):
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]
        return self.label_encoder.transform(new_data_list)
    
def predict(data):
    
#     print(data)
    # Clear Date
    data['clear_date'] = pd.to_datetime(data['clear_date'], format = '%Y-%m-%d')
    # Posting Date
    data['posting_date'] = pd.to_datetime(data['posting_date'], format = '%Y-%m-%d')
    # Due in Date
    data['due_in_date'] = pd.to_datetime(data['due_in_date'], format = '%Y-%m-%d')
    # Baseline Create Date
    data['baseline_create_date'] = pd.to_datetime(data['baseline_create_date'], format = '%Y-%m-%d')
    data['converted_usd'] = pd.to_numeric(data['converted_usd'])
     # buisness_year
    data['buisness_year'] = pd.to_datetime(data['buisness_year'], format = '%Y-%m-%d')
    data['buisness_year'] = data['buisness_year'].dt.year
    
    from sklearn.preprocessing import LabelEncoder
    business_codern = LabelEncoder()
    business_codern.fit(data['business_code'])
    data['business_code_enc'] = business_codern.transform(data['business_code'])
    data['cust_number'] = data['cust_number'].replace('CCCA02',"1").replace(['CCU001','CCU013','CCU002'],"2").replace(['CC3411','CC6000'],"3").astype(int)
    data['day_of_cleardate'] = data['clear_date'].dt.day
    data['month_of_cleardate'] = data['clear_date'].dt.month
    data['year_of_cleardate'] = data['clear_date'].dt.year

    data['day_of_postingdate'] = data['posting_date'].dt.day
    data['month_of_postingdate'] = data['posting_date'].dt.month
    data['year_of_postingdate'] = data['posting_date'].dt.year

    data['day_of_due'] = data['due_in_date'].dt.day
    data['month_of_due'] = data['due_in_date'].dt.month
    data['year_of_due'] = data['due_in_date'].dt.year

    data['day_of_createdate'] = data['baseline_create_date'].dt.day
    data['month_of_createdate'] = data['baseline_create_date'].dt.month
    data['year_of_createdate'] = data['baseline_create_date'].dt.year
    label_encoder = EncoderExt()
    label_encoder1 = EncoderExt()
    label_encoder1.fit(data['cust_payment_terms'])
    label_encoder1.fit(data['business_code'])
    label_encoder.fit(data['name_customer'])
    data['cust_payment_terms_enc']=label_encoder1.transform(data['cust_payment_terms'])
    data['business_code_enc']=label_encoder1.transform(data['business_code'])
    data['name_customer_enc']=label_encoder.transform(data['name_customer'])
    data.drop(['business_code',"baseline_create_date","due_in_date","posting_date","name_customer","clear_date","cust_payment_terms"],axis=1,inplace=True)
    data.drop(['day_of_cleardate',"month_of_cleardate","year_of_cleardate"],axis=1,inplace=True)
    data2=data[['cust_number', 'buisness_year', 'doc_id', 'converted_usd',
           'business_code_enc', 'name_customer_enc', 'cust_payment_terms_enc',
           'day_of_postingdate', 'month_of_postingdate', 'year_of_postingdate',
           'day_of_createdate', 'month_of_createdate', 'year_of_createdate',
           'day_of_due', 'month_of_due', 'year_of_due']]
    import pickle
    model = pickle.load(open("model.sav", 'rb'))
    final_result = model.predict(data2)
    final_result = pd.Series(final_result,name='avg_delay')
    data2.reset_index(drop=True,inplace=True)
    Final = data2.merge(final_result , on = data.sl_no )
    Final['avg_delay'] = Final.apply(lambda row: row.avg_delay//(24 * 3600), axis = 1)
    bins= [-np.inf, 0, 15, 30, 45, 60, 100]
    labels = ['<0','0-15','16-30','31-45','46-60','Greatar than 60']
    Final['Aging Bucket'] = pd.cut(Final['avg_delay'], bins=bins, labels=labels, right=False)
    Final.drop(['key_0',"avg_delay"],axis=1,inplace=True)
    Final1=Final[['doc_id','Aging Bucket']].copy()
    doc = Final1["doc_id"].tolist()
    aging= Final1["Aging Bucket"].tolist()   
    a=[]
    for i in range(len(doc)):
        a.append({"doc_id":doc[i],"aging_bucket": str(aging[i])})
    

    return a



@app.route("/predict",methods=["POST", "GET"])
def home():
    if request.method == "POST":
#         print(request.json)
        content = request.get_json()
        required_cols = ["sl_no", "business_code", "cust_number", "name_customer", "clear_date", "buisness_year", "doc_id", "posting_date", "due_in_date", "baseline_create_date", "cust_payment_terms", "total_open_amount"]
        rows = []
        
        for row in content:
            row_data = []
            for key, value in row.items():
                if key in required_cols:
                    row_data.append(value)
            rows.append(row_data)
#         print(rows)            
        data = pd.DataFrame(rows, columns=["sl_no", "business_code", "cust_number", "name_customer", "clear_date", "buisness_year", "doc_id", "posting_date", "due_in_date", "converted_usd", "baseline_create_date", "cust_payment_terms"])
        data.set_index('sl_no')
        
        data['doc_id'] = data['doc_id'].astype(int)
#         data['converted_usd'] = data['converted_usd'].astype(float)
#       print(data.info())
#       print(data)
#         print(predict(data))   
        response = make_response(jsonify(predict(data)),200)
        response.headers["Access-Control-Allow-Origin"]="*"
        response.headers["Content-Type"] = "application/json"
        return response
   
        
if __name__ == "__main__":
    app.run()
    



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2022 11:19:43] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\abhis\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
127.0.0.1 - - [19/Apr/2022 11:19:43] "POST /predict HTTP/1.1" 200 -


       doc_id Aging Bucket
0  1930438491           <0


127.0.0.1 - - [19/Apr/2022 11:31:32] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\abhis\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
127.0.0.1 - - [19/Apr/2022 11:31:32] "POST /predict HTTP/1.1" 200 -


       doc_id Aging Bucket
0  1930438491           <0


...

       doc_id Aging Bucket
0  1930438491           <0


127.0.0.1 - - [20/Apr/2022 09:10:45] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\abhis\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
127.0.0.1 - - [20/Apr/2022 09:10:45] "POST /predict HTTP/1.1" 200 -


       doc_id Aging Bucket
0  1929646410         0-15


127.0.0.1 - - [20/Apr/2022 09:10:53] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\abhis\anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
127.0.0.1 - - [20/Apr/2022 09:10:53] "POST /predict HTTP/1.1" 200 -


       doc_id Aging Bucket
0  1929646410         0-15
